## Задание 1

**Написать теггер на данных с руским языком**
1. проверить UnigramTagger, BigramTagger, TrigramTagger и их комбмнации
2. написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов
3. сравнить все реализованные методы сделать выводы


## загрузка данных

In [1]:
!pip install pyconll

In [2]:
import pyconll

In [3]:
!mkdir datasets

In [4]:
!wget -O ./datasets/ru_syntagrus-ud-train.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train.conllu
!wget -O ./datasets/ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu

--2021-07-29 06:08:35--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 81039282 (77M) [text/plain]
Saving to: ‘./datasets/ru_syntagrus-ud-train.conllu’

./datasets/ru_synta 100%[===================>]  77.28M   136MB/s    in 0.6s    

2021-07-29 06:08:37 (136 MB/s) - ‘./datasets/ru_syntagrus-ud-train.conllu’ saved [81039282/81039282]

--2021-07-29 06:08:37--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercont

In [5]:
import pyconll

import warnings
warnings.filterwarnings("ignore")

import nltk
from nltk.corpus import brown
from nltk.tag import DefaultTagger
from nltk.tag import UnigramTagger
from nltk.tag import BigramTagger, TrigramTagger

from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

from sklearn.preprocessing import LabelEncoder

In [6]:
full_train = pyconll.load_from_file('datasets/ru_syntagrus-ud-train.conllu')
full_test = pyconll.load_from_file('datasets/ru_syntagrus-ud-dev.conllu')

In [10]:
for sent in full_train[:2]:
    for token in sent:
        print(token.form, token.upos)
    print()

Анкета NOUN
. PUNCT

Начальник NOUN
областного ADJ
управления NOUN
связи NOUN
Семен PROPN
Еремеевич PROPN
был AUX
человек NOUN
простой ADJ
, PUNCT
приходил VERB
на ADP
работу NOUN
всегда ADV
вовремя ADV
, PUNCT
здоровался VERB
с ADP
секретаршей NOUN
за ADP
руку NOUN
и CCONJ
иногда ADV
даже PART
писал VERB
в ADP
стенгазету NOUN
заметки NOUN
под ADP
псевдонимом NOUN
" PUNCT
Муха NOUN
" PUNCT
. PUNCT



In [11]:
fdata_train = []
for sent in full_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])
    
fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])
    
fdata_sent_test = []
for sent in full_test[:]:
    fdata_sent_test.append([token.form for token in sent])
    
MAX_SENT_LEN = max(len(sent) for sent in full_train)
MAX_ORIG_TOKEN_LEN = max(len(token.form) for sent in full_train for token in sent)
print('Наибольшая длина предложения', MAX_SENT_LEN)
print('Наибольшая длина токена', MAX_ORIG_TOKEN_LEN)

Наибольшая длина предложения 205
Наибольшая длина токена 47


In [12]:
all_train_texts = [' '.join(token.form for token in sent) for sent in full_train]
all_test_texts = [' '.join(token.form for token in sent) for sent in full_test]

all_train_labels = [' '.join(token.form for token in sent) for sent in full_train]
all_test_labels = [' '.join(token.form for token in sent) for sent in full_test]
print('\n'.join(all_train_texts[:10]))

Анкета .
Начальник областного управления связи Семен Еремеевич был человек простой , приходил на работу всегда вовремя , здоровался с секретаршей за руку и иногда даже писал в стенгазету заметки под псевдонимом " Муха " .
В приемной его с утра ожидали посетители , - кое-кто с важными делами , а кое-кто и с такими , которые легко можно было решить в нижестоящих инстанциях , не затрудняя Семена Еремеевича .
Однако стиль работы Семена Еремеевича заключался в том , чтобы принимать всех желающих и лично вникать в дело .
Приемная была обставлена просто , но по-деловому .
У двери стоял стол секретарши , на столе - пишущая машинка с широкой кареткой .
В углу висел репродуктор и играло радио для развлечения ожидающих и еще для того , чтобы заглушать голос начальника , доносившийся из кабинета , так как , бесспорно , среди посетителей могли находиться и случайные люди .
Кабинет отличался скромностью , присущей Семену Еремеевичу .
В глубине стоял широкий письменный стол с бронзовыми чернильницами

DefaultTagger


In [13]:
default_tagger = nltk.DefaultTagger('NOUN')

default_tagger.evaluate(fdata_test)

0.23568564014423887


UnigramTagger

In [14]:
unigram_tagger = UnigramTagger(fdata_train)
unigram_tagger.evaluate(fdata_test)

0.8772537323492737

BigramTagger

In [15]:
bigram_tagger = BigramTagger(fdata_train, backoff=unigram_tagger)
bigram_tagger.evaluate(fdata_test)

0.8829828463586425


TrigramTagger

In [16]:
trigram_tagger = TrigramTagger(fdata_train, backoff=bigram_tagger)
trigram_tagger.evaluate(fdata_test)

0.882081353418933

In [17]:
trigram_tagger.tag(fdata_sent_test[1])

[('В', 'ADP'),
 ('старой', 'ADJ'),
 ('трактовке', 'NOUN'),
 ('вместо', 'ADP'),
 ('слова', 'NOUN'),
 ('"', 'PUNCT'),
 ('порядок', 'NOUN'),
 ('"', 'PUNCT'),
 ('использовалось', 'VERB'),
 ('слово', 'NOUN'),
 ('"', 'PUNCT'),
 ('последовательность', 'NOUN'),
 ('"', 'PUNCT'),
 (',', 'PUNCT'),
 ('но', 'CCONJ'),
 ('по', 'ADP'),
 ('мере', 'NOUN'),
 ('развития', 'NOUN'),
 ('параллельности', None),
 ('в', 'ADP'),
 ('работе', 'NOUN'),
 ('компьютеров', 'NOUN'),
 ('слово', 'NOUN'),
 ('"', 'PUNCT'),
 ('последовательность', 'NOUN'),
 ('"', 'PUNCT'),
 ('стали', 'VERB'),
 ('заменять', 'VERB'),
 ('более', 'ADV'),
 ('общим', 'ADJ'),
 ('словом', 'NOUN'),
 ('"', 'PUNCT'),
 ('порядок', 'NOUN'),
 ('"', 'PUNCT'),
 ('.', 'PUNCT')]

Комбинация тэггеров

In [18]:
def backoff_tagger(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff


backoff = DefaultTagger('NOUN') 
tag = backoff_tagger(fdata_train,  
                     [UnigramTagger, BigramTagger, TrigramTagger],  
                     backoff = backoff) 
  
tag.evaluate(fdata_test)

0.9119991237825633

Векторайзеры

In [19]:
train_tok = []
train_label = []
for sent in fdata_train[:]:
    for tok in sent:
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])
        
test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        test_tok.append(tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

In [20]:
le = LabelEncoder()

train_enc_labels = le.fit_transform(train_label)
test_enc_labels = le.transform(test_label)

print(f'Classes: {le.classes_} \nNumber of classes: {len(le.classes_)}')

Classes: ['ADJ' 'ADP' 'ADV' 'AUX' 'CCONJ' 'DET' 'INTJ' 'NOUN' 'NO_TAG' 'NUM' 'PART'
 'PRON' 'PROPN' 'PUNCT' 'SCONJ' 'SYM' 'VERB' 'X'] 
Number of classes: 18


In [21]:
for vectorizer in [CountVectorizer, HashingVectorizer, TfidfVectorizer]:

    scaler = StandardScaler(with_mean=False)
    coder = vectorizer(ngram_range=(1, 5), analyzer='char')
    

    X_train = coder.fit_transform(train_tok)
    X_test = coder.transform(test_tok)
    
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)    
    
    lr = LogisticRegression(random_state=0, max_iter = 100, n_jobs=7)
    lr.fit(X_train, train_enc_labels)

    pred = lr.predict(X_test)

    print(vectorizer, accuracy_score(test_enc_labels, pred))

<class 'sklearn.feature_extraction.text.CountVectorizer'> 0.943644053516665
<class 'sklearn.feature_extraction.text.HashingVectorizer'> 0.9471826239342163
<class 'sklearn.feature_extraction.text.TfidfVectorizer'> 0.9487749806221144


In [22]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('russian')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [23]:
for vectorizer in [CountVectorizer, HashingVectorizer, TfidfVectorizer]:

    scaler = StandardScaler(with_mean=False)
    coder = vectorizer(ngram_range=(1, 5), analyzer='word', stop_words=stop_words)
    

    X_train = coder.fit_transform(train_tok)
    X_test = coder.transform(test_tok)
    
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)    
    
    lr = LogisticRegression(random_state=0, max_iter = 100, n_jobs=7)
    lr.fit(X_train, train_enc_labels)

    pred = lr.predict(X_test)

    print(vectorizer, accuracy_score(test_enc_labels, pred))

<class 'sklearn.feature_extraction.text.CountVectorizer'> 0.622190206585111
<class 'sklearn.feature_extraction.text.HashingVectorizer'> 0.6241448454824251
<class 'sklearn.feature_extraction.text.TfidfVectorizer'> 0.6216762713578



Из одиночных теггеров лучший результат показали BigramTagger (0.88298) и TrigramTagger (0.88208) теггеры.
Комбинация теггеров [UnigramTagger, BigramTagger, TrigramTagger] дает лучший результат, чем использовать теггеры  по отдельности 0.911
Векторайзеры с символьными н-граммами работают лучше теггеров. Лучший скор показывает TfidfVectorizer (0.9489).

# Задание 2

много дополнительных датасетов на русском языке

https://natasha.github.io/corus/  
https://github.com/natasha/corus

мы будем использовать данные http://www.labinform.ru/pub/named_entities/

**Проверить насколько хорошо работает NER**

1. взять нер из nltk
2. проверить deeppavlov
3. написать свой нер попробовать разные подходы:
* передаём в сетку токен и его соседей
* передаём в сетку только токен

4. сделать выводы по вашим экспериментам какой из подходов успешнее справляется


при обучении своего нера незабудьте разделить выборку

In [24]:
!pip install corus

     |████████████████████████████████| 83 kB 998 kB/s 


In [25]:
import corus

In [26]:
!wget http://www.labinform.ru/pub/named_entities/collection5.zip

--2021-07-29 06:48:30--  http://www.labinform.ru/pub/named_entities/collection5.zip
Resolving www.labinform.ru (www.labinform.ru)... 80.240.100.4
Connecting to www.labinform.ru (www.labinform.ru)|80.240.100.4|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1899530 (1.8M) [application/zip]
Saving to: ‘collection5.zip’

collection5.zip     100%[===================>]   1.81M   371KB/s    in 5.7s    

2021-07-29 06:48:37 (323 KB/s) - ‘collection5.zip’ saved [1899530/1899530]



In [ ]:
!unzip collection5.zip

In [28]:
!rm collection5.zip

In [29]:
!ls

Collection5  datasets  sample_data


In [30]:
from corus import load_ne5

dir = 'Collection5/'
records = load_ne5(dir)
next(records)


Ne5Markup(
    id='288',
    text='Д.Медведев уволил пять генералов МВД.\r\n\r\nПрезидент России Дмитрий Медведев своим указом освободил от должности ряд высокопоставленных сотрудников органов внутренних дел Российской Федерации. Как сообщили в пресс-службе Кремля, своим решением глава государства снял с должности начальника управления внутренних дел по Калининградской области генерал-майора милиции Сергея Кириченко.\r\n\r\nТакже своих должностей лишились: заместитель начальника Главного управления МВД РФ по Северо-Западному федеральному округу генерал-майор юстиции Сергей Панфилов; заместитель начальника Главного управления внутренних дел по Алтайскому краю генерал-майор юстиции Виктор Баранов; заместитель начальника Главного управления внутренних дел по Пермскому краю по экономической безопасности генерал-майор милиции Юрий Савченко и заместитель начальника Главного управления МВД РФ по Южному федеральному округу генерал-майор юстиции Александр Шашин.',
    spans=[Ne5Span(
         i

процедуры обработки взять из вебинарного ноутбука

In [31]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [32]:
document = next(records).text

nltk.pos_tag(nltk.word_tokenize(document))

[('Акционеры', 'NN'),
 ('``', '``'),
 ('Нутритека', 'JJ'),
 ("''", "''"),
 ('на', 'NN'),
 ('внеочередном', 'NNP'),
 ('собрании', 'NNP'),
 ('изберут', 'NNP'),
 ('новый', 'NNP'),
 ('совет', 'NNP'),
 ('директоров', 'NNP'),
 ('.', '.'),
 ('Совет', 'VB'),
 ('директоров', 'JJ'),
 ('ОАО', 'NNP'),
 ('``', '``'),
 ('Нутринвестхолдинг', 'NN'),
 ("''", "''"),
 ('(', '('),
 ('группа', 'FW'),
 ('``', '``'),
 ('Нутритек', 'FW'),
 ("''", "''"),
 (')', ')'),
 ('созывает', 'FW'),
 ('сегодня', 'FW'),
 ('внеочередное', 'FW'),
 ('общее', 'FW'),
 ('собрание', 'FW'),
 ('акционеров', 'FW'),
 ('.', '.'),
 ('Список', 'FW'),
 ('лиц', 'NN'),
 (',', ','),
 ('имеющих', 'NNP'),
 ('право', 'NNP'),
 ('на', 'NNP'),
 ('участие', 'NNP'),
 ('во', 'NNP'),
 ('внеочередном', 'NNP'),
 ('общем', 'NNP'),
 ('собрании', 'NNP'),
 ('акционеров', 'NNP'),
 (',', ','),
 ('утвержден', 'NNP'),
 ('по', 'NNP'),
 ('состоянию', 'NNP'),
 ('на', 'VBD'),
 ('21', 'CD'),
 ('апреля', 'JJ'),
 ('2011г', 'CD'),
 ('.', '.'),
 ('Акционерам', 'VB'),
 

In [33]:
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [34]:
{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(document))) if hasattr(chunk, 'label') }

{('Partners Сергей Азатян', 'ORGANIZATION'),
 ('Акционеры', 'GPE'),
 ('Александр Рогачев', 'PERSON'),
 ('Алексей Махнев', 'PERSON'),
 ('Артур Микаелян', 'PERSON'),
 ('Геннадий Шамраев', 'PERSON'),
 ('Дмитрий Скуратов', 'PERSON'),
 ('Константин Джимбинов', 'ORGANIZATION'),
 ('Константин Малофеев', 'PERSON'),
 ('Кроме', 'PERSON'),
 ('Максим Волков', 'PERSON'),
 ('Напомним', 'PERSON'),
 ('ОАО', 'ORGANIZATION'),
 ('Олег Хуажев', 'PERSON'),
 ('Помимо', 'PERSON'),
 ('Ранее', 'PERSON'),
 ('Сажинов', 'PERSON'),
 ('Эстонии', 'PERSON')}

Spacy

In [ ]:
import spacy
from spacy import displacy
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_md-2.2.0/en_core_web_md-2.2.0.tar.gz
import en_core_web_md

In [52]:
nlp = en_core_web_md.load()
doc = nlp(document)

displacy.render(doc, style="dep") # (1)
displacy.render(doc, style="ent") # (2)

'<div class="entities" style="line-height: 2.5; direction: ltr">\n<mark class="entity" style="background: #7aecec; padding: 0.45em 0.6em; margin: 0 0.25em; line-height: 1; border-radius: 0.35em;">\n    Акционеры\n    <span style="font-size: 0.8em; font-weight: bold; line-height: 1; border-radius: 0.35em; text-transform: uppercase; vertical-align: middle; margin-left: 0.5rem">ORG</span>\n</mark>\n &quot;Нутритека&quot; на внеочередном собрании изберут новый \n<mark class="entity" style="background: #aa9cfc; padding: 0.45em 0.6em; margin: 0 0.25em; line-height: 1; border-radius: 0.35em;">\n    совет директоров\n    <span style="font-size: 0.8em; font-weight: bold; line-height: 1; border-radius: 0.35em; text-transform: uppercase; vertical-align: middle; margin-left: 0.5rem">PERSON</span>\n</mark>\n.\r</br>\r</br>\n<mark class="entity" style="background: #aa9cfc; padding: 0.45em 0.6em; margin: 0 0.25em; line-height: 1; border-radius: 0.35em;">\n    Совет директоров ОАО\n    <span style="fo

In [36]:
pip install razdel

In [37]:
from razdel import tokenize

In [53]:
words_docs = []
for ix, rec in enumerate(records):
    words = []
    for token in tokenize(rec.text):
       
        result = 'None'        
        
        for item in rec.spans:            
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'PER'):
                result = 'PER'
                break
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'ORG'):
                result = 'ORG'
                break
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'MEDIA'):
                result = 'MEDIA'
                break
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'LOC'):
                result = 'LOC'
                break
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'GEOPOLIT'):
                result = 'GEOPOLIT'
                break
                
    
        words.append([token.text, result])
    words_docs.extend(words)

In [54]:
import pandas as pd

In [55]:
df_words = pd.DataFrame(words_docs, columns=['word', 'tag'])


In [56]:
df_words['tag'].value_counts()

None        218810
PER          21145
ORG          13613
LOC           4556
GEOPOLIT      4344
MEDIA         2482
Name: tag, dtype: int64

In [57]:
df_words.head(3)

,word,tag
0,ФАС,ORG
1,России,GEOPOLIT
2,:,None


In [58]:
df_words.shape

(264950, 2)

In [59]:
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, GlobalMaxPooling1D, Conv1D, Conv2D, GRU, LSTM, Dropout, Input
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [60]:
from sklearn import model_selection, preprocessing, linear_model

train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df_words['word'], df_words['tag'])

# labelEncode целевую переменную
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [61]:
train_x.apply(len).max(axis=0)

55

In [62]:
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y))
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y))

train_data = train_data.batch(16)
valid_data = valid_data.batch(16)

In [63]:
AUTOTUNE = tf.data.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
valid_data = valid_data.cache().prefetch(buffer_size=AUTOTUNE)

In [64]:
def custom_standardization(input_data):
    return input_data

def data_preparation(train_data, seq_len=1, vocab_size = 30000):    

    vectorize_layer = TextVectorization(
        standardize=custom_standardization,
        max_tokens=vocab_size,
        output_mode='int',
        output_sequence_length=seq_len)


    # Make a text-only dataset (no labels) and call adapt to build the vocabulary.
    text_data = train_data.map(lambda x, y: x)
    vectorize_layer.adapt(text_data)
    return vectorize_layer

In [65]:
embedding_dim = 64

In [66]:
class modelNER(tf.keras.Model):
    def __init__(self):
        super(modelNER, self).__init__()
        self.emb = Embedding(vocab_size, embedding_dim)
        self.gPool = GlobalMaxPooling1D()
        self.fc1 = Dense(300, activation='relu')
        self.fc2 = Dense(50, activation='relu')
        self.fc3 = Dense(6, activation='softmax')

    def call(self, x):
        x = vectorize_layer(x)
        x = self.emb(x)
        pool_x = self.gPool(x)
        
        fc_x = self.fc1(pool_x)
        fc_x = self.fc2(fc_x)
        
        concat_x = tf.concat([pool_x, fc_x], axis=1)
        prob = self.fc3(concat_x)
        return prob

In [67]:
vocab_size = 30000
vectorize_layer = data_preparation(train_data, seq_len = 1, vocab_size=vocab_size)


mmodel = modelNER()
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
mmodel.fit(train_data, validation_data=valid_data, epochs=5)

Epoch 1/5
12420/12420 [==============================] - 315s 25ms/step - loss: 0.2847 - accuracy: 0.9175 - val_loss: 0.2009 - val_accuracy: 0.9404
Epoch 2/5
12420/12420 [==============================] - 292s 24ms/step - loss: 0.1226 - accuracy: 0.9632 - val_loss: 0.2032 - val_accuracy: 0.9418
Epoch 3/5
12420/12420 [==============================] - 304s 24ms/step - loss: 0.1083 - accuracy: 0.9654 - val_loss: 0.2302 - val_accuracy: 0.9408
Epoch 4/5
12420/12420 [==============================] - 280s 23ms/step - loss: 0.1033 - accuracy: 0.9662 - val_loss: 0.2777 - val_accuracy: 0.8848
Epoch 5/5
12420/12420 [==============================] - 277s 22ms/step - loss: 0.1007 - accuracy: 0.9667 - val_loss: 0.3143 - val_accuracy: 0.8743


In [68]:
vocab_size = 30000
vectorize_layer = data_preparation(train_data, seq_len = 5, vocab_size=vocab_size)


mmodel = modelNER()
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
mmodel.fit(train_data, validation_data=valid_data, epochs=5)

Epoch 1/5
12420/12420 [==============================] - 269s 22ms/step - loss: 0.2949 - accuracy: 0.9140 - val_loss: 0.2048 - val_accuracy: 0.9393
Epoch 2/5
12420/12420 [==============================] - 259s 21ms/step - loss: 0.1257 - accuracy: 0.9623 - val_loss: 0.2008 - val_accuracy: 0.9419
Epoch 3/5
12420/12420 [==============================] - 251s 20ms/step - loss: 0.1094 - accuracy: 0.9652 - val_loss: 0.2127 - val_accuracy: 0.9422
Epoch 4/5
12420/12420 [==============================] - 255s 21ms/step - loss: 0.1043 - accuracy: 0.9661 - val_loss: 0.2475 - val_accuracy: 0.9416
Epoch 5/5
12420/12420 [==============================] - 256s 21ms/step - loss: 0.1015 - accuracy: 0.9665 - val_loss: 0.2403 - val_accuracy: 0.9360


In [69]:
vocab_size = 30000
vectorize_layer = data_preparation(train_data, seq_len = 10, vocab_size=vocab_size)


mmodel = modelNER()
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
mmodel.fit(train_data, validation_data=valid_data, epochs=5)

Epoch 1/5
12420/12420 [==============================] - 275s 22ms/step - loss: 0.2935 - accuracy: 0.9146 - val_loss: 0.2023 - val_accuracy: 0.9396
Epoch 2/5
12420/12420 [==============================] - 271s 22ms/step - loss: 0.1255 - accuracy: 0.9621 - val_loss: 0.3328 - val_accuracy: 0.8956
Epoch 3/5
12420/12420 [==============================] - 256s 21ms/step - loss: 0.1100 - accuracy: 0.9653 - val_loss: 0.3002 - val_accuracy: 0.8956
Epoch 4/5
12420/12420 [==============================] - 266s 21ms/step - loss: 0.1042 - accuracy: 0.9662 - val_loss: 0.3052 - val_accuracy: 0.8940
Epoch 5/5
12420/12420 [==============================] - 263s 21ms/step - loss: 0.1014 - accuracy: 0.9667 - val_loss: 0.3335 - val_accuracy: 0.8947


Результаты оказались чуть хуже при длине последовательности 10, при осталбных - примерно одинаковые